In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
import numpy as np
import pandas as pd
import geopandas as gpd
from math import radians, cos, sin, asin, sqrt, atan2
import matplotlib.pyplot as plt
%matplotlib inline
import networkx as nx

from collections import Counter
from datetime import datetime
from pyproj import Proj, transform

import rasterio
from rasterio.transform import xy
from rasterio.plot import show
from rasterio.features import geometry_mask

from shapely.geometry import Point, Polygon, LineString, MultiLineString

import statsmodels.formula.api as smf
import tqdm.notebook as tqdm

import seaborn as sns
# from osgeo import gdal
sns.set()
%config InlineBackend.figure_format = 'retina'

In [2]:
data_path = '/home/gleb/Desktop/thesis/data/'

countries_path = data_path + 'world-administrative-boundaries/'
basins_6_path = data_path + 'HydroBASINS Africa 6 level/'
basins_12_path = data_path + 'HydroBASINS Africa 12 level/'
conflict_path = data_path + 'conflicts/'
dams_path = data_path + 'GRanD_Version_1_3/'
sheds_file_3 = data_path + 'hyd_af_dem_3s/af_dem_3s.tif'
sheds_file_15 = data_path + 'hyd_af_dem_15s/hyd_af_dem_15s.tif'
rivers_file = data_path + 'HydroRIVERS_v10_af/HydroRIVERS_v10_af.gdb'

output_path = '/home/gleb/Desktop/thesis/outcomes/'

In [9]:
df = gpd.read_file(output_path + '5rivers_fully_prepared_data_all_conflicts.shp')

df1 = gpd.read_file(conflict_path + '1997-01-01-2023-09-30.csv', sep=';', dtype={'timestamp': 'object'})
gdf_conf = gpd.GeoDataFrame(df1, geometry = gpd.points_from_xy(df1.longitude, df1.latitude))
gdf_conf.set_crs('epsg:4326', inplace=True)

# Filter by region:
regions = ['Eastern Africa', 'Middle Africa', 'Northern Africa', 'Southern Africa', 'Western Africa']
gdf_conf = gdf_conf[gdf_conf['region'].apply(lambda x: True if x in regions else False)]
gdf_conf.reset_index(drop=True, inplace=True)

# Filter by battles:
battles_gdf = gdf_conf[gdf_conf.event_type == 'Battles']
battles_gdf.reset_index(drop=True, inplace=True)
battles_gdf = gdf_conf.copy()

del gdf_conf
del df1

,event_id_cnty,event_date,year,time_precision,disorder_type,event_type,sub_event_type,actor1,assoc_actor_1,inter1,...,latitude,longitude,geo_precision,source,source_scale,notes,fatalities,tags,timestamp,geometry
0,BFO10686,30 September 2023,2023,1,Political violence,Battles,Armed clash,JNIM: Group for Support of Islam and Muslims,,2,...,13.8478,-2.4172,1,Whatsapp,New media,"On 30 September 2023, presumed JNIM militants ...",0,,1696869398,POINT (-2.41720 13.84780)
1,BFO10693,30 September 2023,2023,1,Political violence,Battles,Armed clash,JNIM: Group for Support of Islam and Muslims,,2,...,12.7301,-4.0967,1,Facebook; Signal; Whatsapp,New media,"On 30 September 2023, presumed JNIM militants ...",0,,1696869398,POINT (-4.09670 12.73010)
2,BFO10708,30 September 2023,2023,1,Political violence,Violence against civilians,Attack,JNIM: Group for Support of Islam and Muslims,,2,...,13.7602,-2.426,1,Facebook,New media,"On 30 September 2023, presumed JNIM militants ...",3,,1696869398,POINT (-2.42600 13.76020)
3,CAO7889,30 September 2023,2023,2,Political violence,Violence against civilians,Attack,Islamic State (West Africa) and/or Boko Haram ...,,2,...,10.7424,13.8023,2,Twitter; Xinhua,New media-International,"Around 30 September 2023, ISWAP or Boko Haram ...",1,,1696869398,POINT (13.80230 10.74240)
4,CDI2876,30 September 2023,2023,1,Political violence,Violence against civilians,Attack,Police Forces of the Ivory Coast (2011-),,1,...,7.4125,-7.5538,1,Koaci News; Soir Info,National,"On 30 September 2023, policemen beat a civilia...",1,,1696869398,POINT (-7.55380 7.41250)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
984432,SIE2,01 January 1997,1997,3,Political violence,Battles,Government regains territory,Military Forces of Sierra Leone (1996-1997),,1,...,8.4642,-10.9332,2,No Peace Without Justice; SL-LED,Local partner-New media,"Around 1 January 1997 (month of), Military For...",0,,1670286851,POINT (-10.93320 8.46420)
984433,SIE3,01 January 1997,1997,3,Political violence,Battles,Armed clash,Kamajor Militia,,3,...,8.1221,-11.7047,2,No Peace Without Justice; SL-LED,Local partner-New media,"Around 1 January 1997 (month of), Kamajor Mili...",0,,1670286851,POINT (-11.70470 8.12210)
984434,SIE6,01 January 1997,1997,3,Political violence,Violence against civilians,Attack,Military Forces of Sierra Leone (1996-1997),,1,...,7.5317,-12.4694,2,No Peace Without Justice; SL-LED,Local partner-New media,"Around 1 January 1997 (month of), Military For...",0,,1670286851,POINT (-12.46940 7.53170)
984435,SIE7,01 January 1997,1997,3,Political violence,Battles,Armed clash,Kamajor Militia,,3,...,8.231,-12.338,2,SL-LED; No Peace Without Justice,Local partner-New media,"Around 1 January 1997 (month of), Kamajor Mili...",0,,1670286851,POINT (-12.33800 8.23100)
